# RoBERTa-based Manipulation Detection (Binary Classification)
This notebook uses `roberta-base` to classify dialogue as manipulative or not using the MentalManip dataset.

In [1]:
!pip install -q transformers
!pip install -q datasets
!pip install -q evaluate
## transformers upgrade
!pip install -q --upgrade transformers

## Datasets need upgrading to work
!pip install -q --upgrade datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 73.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import classification_report


In [3]:
# Load the MentalManip dataset (binary classification)
# Load dataset
dataset = load_dataset("audreyeleven/MentalManip", name="mentalmanip_maj")

print(dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Some datasets params were ignored: ['license']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


mentalmanip_maj.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/4000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'manipulative', 'technique', 'vulnerability'],
        num_rows: 4000
    })
})


In [4]:
# Ensure the 'manipulative' column is class-labeled
dataset = dataset.class_encode_column("manipulative")




Stringifying the column:   0%|          | 0/4000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [5]:
model_ckpt = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def tokenize_fn(example):
    return tokenizer(example["dialogue"], truncation=True, padding="max_length", max_length=128)

tokenized = dataset.map(tokenize_fn, batched=True)
print(tokenized)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'manipulative', 'technique', 'vulnerability', 'input_ids', 'attention_mask'],
        num_rows: 4000
    })
})


In [6]:
# Split the dataset into training and testing sets
train_test_split = tokenized["train"].train_test_split(test_size=0.2) # Adjust the test_size as needed

# Update the tokenized dataset with the new splits
tokenized["train"] = train_test_split["train"]
tokenized["test"] = train_test_split["test"]

# Make the data work with the nomenclature
tokenized = tokenized.rename_column("manipulative", "labels")

print(tokenized)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'labels', 'technique', 'vulnerability', 'input_ids', 'attention_mask'],
        num_rows: 3200
    })
    test: Dataset({
        features: ['id', 'dialogue', 'labels', 'technique', 'vulnerability', 'input_ids', 'attention_mask'],
        num_rows: 800
    })
})


In [7]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt,
    num_labels=2  # Binary classification
)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:

# Install PEFT if not already installed
!pip install -q peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.6 MB/s eta 0:00:00


In [9]:

# Import LoRA-specific modules
from peft import LoraConfig, get_peft_model, TaskType

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Print trainable parameters to confirm LoRA setup
model.print_trainable_parameters()


trainable params: 887,042 || all params: 125,534,212 || trainable%: 0.7066


In [10]:
## Training args

training_args = TrainingArguments(
    output_dir="./bert-binary-manip",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    run_name="bert-binary-manip",
    report_to="none",
)


In [11]:
## Evaluation metrics

import evaluate
import numpy as np

accuracy = evaluate.load('accuracy')
precision = evaluate.load('precision')
recall = evaluate.load('recall')
f1 = evaluate.load('f1')

def compute_metrics(p):
    predictions, labels = p
    predictions_argmax = np.argmax(predictions, axis=1)

    return {
        "accuracy": accuracy.compute(predictions=predictions_argmax, references=labels)["accuracy"],
        "precision": precision.compute(predictions=predictions_argmax, references=labels, average='weighted')["precision"],
        "recall": recall.compute(predictions=predictions_argmax, references=labels, average='weighted')["recall"],
        "f1": f1.compute(predictions=predictions_argmax, references=labels, average='weighted')["f1"],
    }

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()



/tmp/ipython-input-499019960.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.651400,0.601993,0.693750,0.481289,0.693750,0.568312
2,0.643400,0.592476,0.685000,0.650110,0.685000,0.653609
3,0.490200,0.613678,0.688750,0.646064,0.688750,0.642377
4,0.457300,0.611381,0.690000,0.655143,0.690000,0.656591


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.

TrainOutput(global_step=800, training_loss=0.5588035422563553, metrics={'train_runtime': 226.8931, 'train_samples_per_second': 56.414, 'train_steps_per_second': 3.526, 'total_flos': 850675354828800.0, 'train_loss': 0.5588035422563553, 'epoch': 4.0})

In [13]:

##  Easy eval results...
##  Get the log history from the trainer state
log_history = trainer.state.log_history

# Initialize placeholders
last_train_loss = None
last_eval_loss = None

# Iterate through log history to find the last recorded train and eval loss
for log in reversed(log_history):
    if last_eval_loss is None and "eval_loss" in log:
        last_eval_loss = log["eval_loss"]
    if last_train_loss is None and "loss" in log:
        last_train_loss = log["loss"]
    if last_train_loss is not None and last_eval_loss is not None:
        break

# Calculate overfitting ratio
if last_train_loss is not None and last_eval_loss is not None:
    ratio = last_train_loss / last_eval_loss
    print(f"Training Loss: {last_train_loss:.5f}")
    print(f"Validation Loss: {last_eval_loss:.5f}")
    print(f"Overfitting Ratio: {ratio:.5f}")
    if ratio < 0.6:
        print("Overfitting detected!")
    else:
        print("No significant overfitting.")
else:
    print("Could not find both training and evaluation loss in log history.")


Training Loss: 0.45730
Validation Loss: 0.61138
Overfitting Ratio: 0.74798
No significant overfitting.


In [14]:
# Predict on test set
preds = trainer.predict(tokenized["test"])
y_pred = preds.predictions.argmax(-1)
y_true = preds.label_ids

# Detailed classification report
print(classification_report(y_true, y_pred, target_names=["non-manipulative", "manipulative"]))


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


                  precision    recall  f1-score   support

non-manipulative       0.47      0.26      0.34       245
    manipulative       0.73      0.87      0.79       555

        accuracy                           0.69       800
       macro avg       0.60      0.57      0.57       800
    weighted avg       0.65      0.69      0.65       800

